In [1]:
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
from ae_model import AeNet
from kdd_oh_set import ohkdd

In [2]:
use_gpu = torch.cuda.is_available()
train_data = ohkdd(use_gpu=use_gpu,test_size=0)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)
print("use_gpu:{}".format(use_gpu))

use_gpu:True


In [3]:
net = AeNet(126, [100, 80, 60, 40, 20, 10, 1, 10, 20, 40, 60, 80, 100])
loss_func = torch.nn.MSELoss()
opt = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
%%time
loss_count = []
if use_gpu:
    net = net.cuda()
    loss_func = loss_func.cuda()
lable = torch.Tensor([1 for x in range(32)])
if use_gpu:
    lable = lable.cuda()
for epoch in range(5):
    for i, (x, _) in enumerate(train_loader):
        batch_x = Variable(x)

        out = net(batch_x)  #模型输出
        # print(out.shape)
        loss = loss_func(out, batch_x)  #损失计算

        opt.zero_grad()  #消除旧值
        loss.backward()  #回传
        opt.step()  #更新模型参数

        if i % 5000 == 0:  #打点输出
            loss_count.append(loss)
            print("{}th{}:\t".format(epoch + 1, i), loss.item())

    torch.save(net, r'aenet_1key')  #存储模型

1th0:	 0.13402718305587769
1th5000:	 0.03246915340423584


In [ ]:
plt.figure('aenet_loss')
plt.plot(loss_count, label='LOSS')
plt.legend()
plt.show()